# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/meta-llama/Llama-4-Scout-17B-16E-Instruct',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/Qwen/Qwen2.5-Omni-7B',
 '/all-hands/openhands-lm-32b-v0.1',
 '/openfree/flux-chatgpt-ghibli-lora',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/open-thoughts/OpenThoughts2-1M',
 '/datasets/virtuoussy/Multi-subject-RLVR',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/g

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
meta-llama/Llama-4-Scout-17B-16E-Instruct
Updated
about 17 hours ago
•
16k
•
492
deepseek-ai/DeepSeek

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [34]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in PDF.\n"
    user_prompt+= '''Structure the document as follows:
    1 - Introduction
    2 - Company Core Mission
    3 - Company Vision
    4 - Sperhead Projects
    5 - Future Plan'''
    user_prompt+= 'Contain the documents in no more than 2 pages A4'
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [35]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in PDF.\nStructure the document as follows:\n    1 - Introduction\n    2 - Company Core Mission\n    3 - Company Vision\n    4 - Sperhead Projects\n    5 - Future PlanContain the documents in no more than 2 pages A4Landing page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmeta-llama/Llama-4-Scout-17B-16E-Instruct\nUpdated\nabout 18 hours ago\n•\n16k\n•\n494\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n11 days ago\n•\n158k\n•\n2.39k\nQwen/Qwen2.5-Omni-7B\nUpdated\n7 day

In [36]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    return response.choices[0].message.content
    


In [37]:

result= create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


In [38]:
display(Markdown(result))

# Hugging Face Company Brochure

## Introduction
Welcome to Hugging Face, the vibrant community where innovators collaborate to build the future of artificial intelligence. As a leading platform in machine learning, we empower developers and researchers to create, share, and collaborate on various AI models and applications seamlessly.

## Company Core Mission
Our mission is to democratize artificial intelligence and machine learning by providing an open-source platform that fosters community collaboration. We aim to break down barriers in AI development, making advanced technology accessible to everyone, regardless of their expertise or background.

## Company Vision
At Hugging Face, we envision a future where AI is used to enhance human capabilities and improve everyday life. We aspire to lead the AI and machine learning community by driving innovation, advocating for responsible AI practices, and supporting a diverse ecosystem of developers around the globe.

## Spearhead Projects
Hugging Face hosts a wide array of innovative projects and tools, such as:
- **Transformers**: Leveraging state-of-the-art models for various tasks in machine learning.
- **Diffusers**: Tools dedicated to running advanced diffusion models.
- **Datasets**: A library for accessing and sharing datasets for any ML tasks, promoting easy collaboration.
- **HuggingChat**: An interactive platform for chatting with cutting-edge language models.
These projects empower over 50,000 organizations globally, including notable names like Google, Microsoft, and Intel.

## Future Plans
Looking ahead, Hugging Face is focused on expanding our offerings by:
- Enhancing our enterprise solutions to provide more robust security and dedicated support.
- Continuing to cultivate and grow our open-source community initiatives.
- Developing more sophisticated tools that will accelerate machine learning processes for professionals and enthusiasts alike.

Join us at Hugging Face as we build the future of AI together!

---

For more information about careers, partnerships, or to explore our robust tools, visit our website at [Hugging Face](https://huggingface.co) and become part of the AI revolution!

In [44]:
while True:
    language=input('Choose destination language ')
    if language != "":
        break

Choose destination language  
Choose destination language  Spanish


In [80]:
system_prompt='You are a skilled translator , the document you are going to produce must be in PDF format'

In [81]:
user_prompt= f'Translate to {language} the following text: {result}'

In [82]:
openaiOllama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [83]:
def translate():
    textPdf= openaiOllama.chat.completions.create(
    model = "llama3.2",
    messages= [{'role': 'system', 'content': system_prompt},
                  {'role':'user','content':user_prompt}]
    )
    return textPdf.choices[0].message.content

In [84]:
user_prompt

'Translate to Spanish the following text: # Hugging Face Company Brochure\n\n## Introduction\nWelcome to Hugging Face, the vibrant community where innovators collaborate to build the future of artificial intelligence. As a leading platform in machine learning, we empower developers and researchers to create, share, and collaborate on various AI models and applications seamlessly.\n\n## Company Core Mission\nOur mission is to democratize artificial intelligence and machine learning by providing an open-source platform that fosters community collaboration. We aim to break down barriers in AI development, making advanced technology accessible to everyone, regardless of their expertise or background.\n\n## Company Vision\nAt Hugging Face, we envision a future where AI is used to enhance human capabilities and improve everyday life. We aspire to lead the AI and machine learning community by driving innovation, advocating for responsible AI practices, and supporting a diverse ecosystem of de

In [85]:
finalBrochure= translate()

In [86]:
display(Markdown(finalBrochure))

Aquí te dejo la traducción al español del texto:

# Brochura de la Compañía Hugging Face

## Introducción
Bienvenidos a Hugging Face, la vibrante comunidad donde los innovadores colaboran para construir el futuro de la inteligencia artificial. Como una plataforma líder en aprendizaje automático, nos permitimos empower a los desarrolladores y investigadores para crear, compartir y colaborar en diferentes modelos y aplicaciones de inteligencia artificial de manera sencilla.

## Misión Corporativa
Nuestra misión es democratizar la inteligencia artificial y el aprendizaje automático proporcionando una plataforma de código abierto que fomenta la colaboración comunitaria. Nos proponemos romper barres en el desarrollo de la inteligencia artificial, lo que hace posible que avanzada tecnología sea accesible a todos independientemente del nivel de expertise o background en ello.

## Visión Corporativa
En Hugging Face, nos vislumbramos un futuro donde la inteligencia artificial se utiliza para mejorar las capacidades humanas y hacer mejor nuestra vida diaria. Nos proponemos liderar la comunidad de inteligencia artificial y aprendizaje automático al impulsar la innovación, abogar por prácticas de inteligencia artificial responsables, y apoyando a una amplia red de desarrolladores en todo el mundo.

## Proyectos Destacados
Hugging Face acoge una amplia gama de proyectos innovadores y herramientas como:
- **Transformers**: Utilizando modelos estatales por adelantado para diversas tareas en aprendizaje automático.
- **Diffusers**: Herramientas dedicadas a correr modelos avanzados de difusión.
- **Datasets**: Un catálogo de acceso y compartición de conjuntos de datos para cualquier tarea de ML, promoviendo colaboración fácil.
- **HuggingChat**: Una plataforma interactiva para charlar con modelos lingüísticos avances.
Estos proyectos empoderan a más de 50.000 organizaciones a nivel global, incluyendo nombres notables como Google, Microsoft y Intel.

## Planes Futuros
En cuanto al futuro, Hugging Face se centra en ampliar sus ofrecimientos expandiendo:
- Nuestras soluciones empresariales para proporcionar seguridad y apoyo dedicado más robusto.
- Continuando en la creación y crecimiento de nuestras iniciativas comunitarias abiertas source.
- Desarrollar herramientas más sofisticadas para acelerar procesos de aprendizaje automático tanto para profesionales como entusiastas.

Únete a nosotros en Hugging Face mientras construimos juntos el futuro de la inteligencia artificial!

---

Para obtener más información sobre posiciones laborales, colaboraciones o explorar nuestros robustos herramientas, visita nuestra página web en [Hugging Face](https://huggingface.co) y une tu participa en la revolución de la inteligencia artificial! 

Recuerda que puedes personalizar el formato del PDF según tus preferencias.

In [109]:
finalBrochure

'Aquí te dejo la traducción al español del texto:\n\n# Brochura de la Compañía Hugging Face\n\n## Introducción\nBienvenidos a Hugging Face, la vibrante comunidad donde los innovadores colaboran para construir el futuro de la inteligencia artificial. Como una plataforma líder en aprendizaje automático, nos permitimos empower a los desarrolladores y investigadores para crear, compartir y colaborar en diferentes modelos y aplicaciones de inteligencia artificial de manera sencilla.\n\n## Misión Corporativa\nNuestra misión es democratizar la inteligencia artificial y el aprendizaje automático proporcionando una plataforma de código abierto que fomenta la colaboración comunitaria. Nos proponemos romper barres en el desarrollo de la inteligencia artificial, lo que hace posible que avanzada tecnología sea accesible a todos independientemente del nivel de expertise o background en ello.\n\n## Visión Corporativa\nEn Hugging Face, nos vislumbramos un futuro donde la inteligencia artificial se uti

In [111]:
with open('brochure.txt','w') as file:
    file.write(finalBrochure)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>